In [1]:
!pip install torch numpy torch torchvision


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet50
import os

# Определение модели
class SimpleResNet(nn.Module):
    def __init__(self):
        super(SimpleResNet, self).__init__()
        self.base_model = resnet50(pretrained=True)
        self.base_model.fc = nn.Identity()  # Удаление последнего слоя классификации

    def forward(self, x):
        return self.base_model(x)

# Инициализация модели и перевод ее на доступное устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleResNet().to(device)
model.eval()

# Подготовка DataLoader
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset_path = r"C:/Users/user/Desktop/госкатлог/train_train_2"  # Путь к вашему датасету
dataset = datasets.ImageFolder(dataset_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Извлечение эмбеддингов
def extract_embeddings(dataloader, model):
    model.eval()
    embeddings = []
    labels = []
    filenames = []  # Для сохранения имен файлов

    with torch.no_grad():
        for i, (images, targets) in enumerate(dataloader):
            images = images.to(device)
            output = model(images)
            embeddings.extend(output.cpu().numpy())
            # Получаем имена файлов для текущей партии
            batch_filenames = [os.path.basename(dataloader.dataset.samples[j][0]) for j in range(i * dataloader.batch_size, min((i + 1) * dataloader.batch_size, len(dataloader.dataset.samples)))]
            labels.extend(batch_filenames)  # Теперь labels будет содержать имена файлов
            filenames.extend(batch_filenames)

    return np.array(embeddings), labels, filenames

embeddings, labels, filenames = extract_embeddings(dataloader, model)

# Вывод размеров полученных массивов для проверки
print("Embeddings shape:", embeddings.shape)
print("Labels shape:", len(labels))
print("Number of filenames:", len(filenames))


c:\Users\user\dataM\MuseumCatalogDataQuality\env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\dataM\MuseumCatalogDataQuality\env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Embeddings shape: (15831, 2048)
Labels shape: 15831
Number of filenames: 15831


In [3]:
import numpy as np

# Сохранение эмбеддингов и меток
np.save('saved_embeddings.npy', embeddings)
np.save('saved_labels.npy', labels)

In [4]:
import numpy as np

# Загрузка файлов с эмбеддингами и именами файлов
embeddings = np.load(r'saved_embeddings.npy')
filenames = np.load(r'saved_labels.npy')

# Вывод информации о содержимом файлов
print("Эмбеддинги:")
print(embeddings)

print("Имена файлов:")
print(filenames)


Эмбеддинги:
[[0.285463   0.7481798  0.2868925  ... 0.23863004 1.1046811  0.03772978]
 [0.21932656 0.5549939  0.12329546 ... 0.15431583 0.44666535 0.20698547]
 [0.32345596 0.12081163 0.1805102  ... 0.316159   0.20825344 0.08155081]
 ...
 [0.3693158  2.6201997  0.625854   ... 0.19025482 0.2819961  0.3149265 ]
 [0.03953381 0.43596387 0.14762373 ... 0.1595026  0.49417347 0.08306289]
 [0.5131302  1.0380177  0.18909748 ... 0.0777268  0.2264141  0.25405177]]
Имена файлов:
['14246331.jpg' '14249267.jpg' '15000074.jpg' ... '7750119.jpg'
 '7750123.jpg' '7750125.jpg']
